# Random forest classifier introduction
* this is  a ensemble bagging method coming from sklearn
* there is risk of overfitting 
* to avoid overfitting the max_depth must not be bigger than 15
* Because it is a bagging method good result is not expected for imbalanced data
* Does not need scaling or normalisation

In [ ]:
#For Kaggle
#date=''

#For WL data
source='WL'
date='20241118'

saveImage=False

print('done')

In [ ]:
import sys
import os
from importlib import reload
fpath = os.path.join('..//scripts')
sys.path.append(fpath)

import warnings
warnings.filterwarnings('ignore')

#loading internal scripts
import frauddetection as fd
import sourcedata as sd
import dataimport as di
import result as resultMd
reload(fd)
reload(sd)
reload(di)
reload(resultMd)

print('done')

In [ ]:
# Hyperparameters Results

n_estimatorsFound=60
max_depthFound=23

print('done')

In [ ]:
import pandas as pd

dfTrx = pd.read_csv('../data/cleaned/'+source+'export'+date+'.csv')
dfTrx.head(5)

# Test with default hyperparameters

In [ ]:
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from datetime import datetime

then= datetime.now()
predictors = fd.getPredictors(dfTrx)

x_train, x_test, y_train, y_test, scaler =fd.split_data(dfTrx,predictors)

modelClf = RandomForestClassifier(random_state=42)
modelClf.fit(x_train, y_train)
predsTrain = modelClf.predict(x_train)
predsTest = modelClf.predict(x_test)

now = datetime.now()
duration= now - then
duration_in_s = duration.total_seconds()
print("Duration ",duration_in_s)
resultMd.update_time_response_result('07-sklearn.ensemble.a-bagging','RandomForest','1-Default', duration_in_s)
f1,mcc,roc=fd.print_scores(y_test, predsTest,'all')
resultMd.update_performance_test_result('07-sklearn.ensemble-a-bagging','RandomForest','1-Default', f1,mcc,roc)

fd.print_scores(y_train, predsTrain,'f1', False)
fd.print_scores(y_test, predsTest,'f1')
fd.show_importance(modelClf, predictors)
fd.show_confusion_matrix(y_test, predsTest)


# Default parameters conclusion
Overfitting probably due to the default parameter max_depth=None

In [ ]:
%%script false

from sklearn.model_selection import KFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import recall_score, precision_score, accuracy_score, balanced_accuracy_score, f1_score
import numpy as np


x_train, x_test, y_train, y_test, scaler =fd.split_data(dfTrx,predictors)
predictors = fd.getPredictors(dfTrx)
train_f1s=[]
test_f1s=[]
range= []
# too low max_depth -> no result
for max_depth in np.arange(8,21,1):
    print('max_depth',max_depth)
    # Instantiate model and k-fold
    modelClf = RandomForestClassifier(random_state=42, max_depth=max_depth)
    modelClf.fit(x_train, y_train)
    train_preds = modelClf.predict(x_train)
    train_f1 = f1_score(y_train,train_preds)

    test_preds = modelClf.predict(x_test)
    test_f1 = f1_score(y_test,test_preds)
    #fd.show_confusion_matrix(y_train, train_preds)

    print("train_f1", train_f1)
    print("test_f1", test_f1)
    print("dif", train_f1-test_f1)
    train_f1s.append(train_f1)
    test_f1s.append(test_f1)
    range.append(max_depth)
    print('-----------------------')

fd.plt_train_test(range, train_f1s, "train", test_f1s,"test")

In [ ]:
%%script false

from sklearn.model_selection import KFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import recall_score, precision_score, accuracy_score, balanced_accuracy_score, f1_score
import numpy as np


x_train, x_test, y_train, y_test, scaler =fd.split_data(dfTrx,predictors)
predictors = fd.getPredictors(dfTrx)
train_f1s=[]
test_f1s =[]
range= []
# too low max_depth -> no result
for n_estimators in np.arange(20,201,20):
    print('n_estimators',n_estimators)
    # Instantiate model and k-fold
    modelClf = RandomForestClassifier(random_state=42, max_depth=15, n_estimators=n_estimators)
    modelClf.fit(x_train, y_train)
    train_preds = modelClf.predict(x_train)
    train_f1 = f1_score(y_train,train_preds)

    test_preds = modelClf.predict(x_test)
    test_f1 = f1_score(y_test,test_preds)
    #fd.show_confusion_matrix(y_train, train_preds)

    print("train_f1", train_f1)
    print("test_f1", test_f1)
    print("dif", train_f1-test_f1)
    train_f1s.append(train_f1)
    test_f1s.append(test_f1)
    range.append(n_estimators)
    print('-----------------------')

fd.plt_train_test(range, train_f1s, "train", test_f1s," test")

In [ ]:
%%script false

from sklearn.model_selection import KFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import recall_score, precision_score, accuracy_score, balanced_accuracy_score, f1_score
import numpy as np


x_train, x_test, y_train, y_test, scaler =fd.split_data(dfTrx,predictors)
predictors = fd.getPredictors(dfTrx)
train_f1s=[]
meanScores =[]
range= []
# too low max_depth -> no result
for max_depth in np.arange(8,21,1):
    print('max_depth',max_depth)
    # Instantiate model and k-fold
    modelClf = RandomForestClassifier(random_state=42, max_depth=max_depth)
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    
    scores = cross_val_score(modelClf, x_train, y_train,verbose=1, scoring="f1", cv=kf)

    modelClf.fit(x_train, y_train)
    train_preds = modelClf.predict(x_train)
    train_f1 = f1_score(y_train,train_preds)
    #fd.show_confusion_matrix(y_train, train_preds)
    print("Cross-validated scores:", scores)
    print("Mean f1:", np.mean(scores))
    print("train_f1", train_f1)
    print("dif", train_f1-np.mean(scores))
    train_f1s.append(train_f1)
    meanScores.append(np.mean(scores))
    range.append(max_depth)
    print('-----------------------')

fd.plt_train_test(range, train_f1s, "train", meanScores," score")

In [ ]:
%%script false


from sklearn.model_selection import KFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import recall_score, precision_score, accuracy_score, balanced_accuracy_score, f1_score
import numpy as np


x_train, x_test, y_train, y_test, scaler =fd.split_data(dfTrx,predictors)
predictors = fd.getPredictors(dfTrx)
# too low max_depth -> no result
train_f1s=[]
meanScores =[]
range =[]
for n_estimators in np.arange(50,301,25):
    print('n_estimators',n_estimators)
    # Instantiate model and k-fold
    modelClf = RandomForestClassifier(random_state=42, max_depth=15, n_estimators=n_estimators)
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    
    scores = cross_val_score(modelClf, x_train, y_train,verbose=1, scoring="f1", cv=kf)

    modelClf.fit(x_train, y_train)
    train_preds = modelClf.predict(x_train)
    train_f1 = f1_score(y_train,train_preds)
    #fd.show_confusion_matrix(y_train, train_preds)
    print("Cross-validated scores:", scores)
    print("Mean f1:", np.mean(scores))
    print("train_f1", train_f1)
    print("dif", train_f1-np.mean(scores))
    train_f1s.append(train_f1)
    meanScores.append(np.mean(scores))
    range.append(n_estimators)
    print('-----------------------')

fd.plt_train_test(range, train_f1s, "train", meanScores," score")


# Scaling choice
with random forest the scaling has no influence

# Hyperparameters tuning

In [ ]:
%%script false

from scipy.stats import randint

modelClf = RandomForestClassifier(random_state=42)
dic_param={
    'n_estimators': randint(40,110),
    'max_depth': randint(10,25)
}
res=fd.hyperparameterSelectionRandomizedSearchCV(modelClf, dic_param, 'f1', dfTrx, predictors, [], None)
print(res)

#[CV 5/5; 10/10] END max_depth=20, n_estimators=106;, score=0.121 total time= 2.8min
#{'max_depth': 22, 'n_estimators': 60}
#0.16552081045379466
#score   0.8500551267916208
#scoref1 0.8500551267916208

In [ ]:
%%script false


modelClf = RandomForestClassifier(random_state=42)

dic_param={
    'n_estimators':[55,60,65],
    'max_depth':[21,22,23,24]
}
res=fd.hyperparameterSelectionGridSearchCV(modelClf, dic_param, 'f1', dfTrx, predictors, [], None)
print(res)

#{'max_depth': 23, 'n_estimators': 60}
#0.1914795461010364
#scoref1 0.8729042725797729

In [ ]:
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from datetime import datetime

then= datetime.now()
predictors = fd.getPredictors(dfTrx)

x_train, x_test, y_train, y_test, scaler =fd.split_data(dfTrx,predictors)

modelClf = RandomForestClassifier(random_state=42)
parameters={'max_depth': max_depthFound, 'n_estimators':n_estimatorsFound}
modelClf.set_params(**parameters)

modelClf.fit(x_train, y_train)
predsTrain = modelClf.predict(x_train)
predsTest = modelClf.predict(x_test)

now = datetime.now()
duration= now - then
duration_in_s = duration.total_seconds()
print("Duration ",duration_in_s)
resultMd.update_time_response_result('07-sklearn.ensemble.a-bagging','RandomForest','2-After tuning', duration_in_s)
f1,mcc,roc=fd.print_scores(y_test, predsTest,'all')
resultMd.update_performance_test_result('07-sklearn.ensemble.a-bagging','RandomForest','2-After tuning', f1,mcc,roc)

fd.print_scores(y_train, predsTrain,'f1', False)
fd.print_scores(y_test, predsTest,'f1')
fd.show_importance(modelClf, predictors)
fd.show_confusion_matrix(y_test, predsTest)
fd.show_prediction_graph(modelClf, x_test,y_test,'../imgs/FD07A-RandomForestClassifierProbaHistogram' if saveImage else None)



In [ ]:
files = fd.getAllFiles()

range = []
f1s = []
rocs = []
loop =0
for file in files:
    loop=loop+1
    range.append(loop)
    print(file)
  
    dfTrx = pd.read_csv('../data/cleaned/'+source+file)
    predictors = fd.getPredictors(dfTrx)

    preds = modelClf.predict(dfTrx[predictors])

    f1,mcc,roc= fd.print_scores(dfTrx['Class'], preds,'All', True)
    #fd.show_importance(modelClf,predictors)
    fd.show_confusion_matrix(dfTrx['Class'], preds)
    f1s.append(f1)
    rocs.append(roc)

fd.plt_train_test(range, f1s)
resultMd.update_performance_nextdays_result('07-sklearn.ensemble.a-bagging','RandomForestClassifier','2-After tuning', f1s[0],f1s[1],f1s[2],f1s[3],rocs[0],rocs[1],rocs[2],rocs[3])



# 